<a href="https://colab.research.google.com/github/robabsmith/CALDISS-SDS-PhD-school/blob/master/Day_04_Portfolio_Part_1_Early_session_Rob_Smith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfolio Day 4: NLP

## Part 1: Mid day do it yourself sections

In [0]:
import re

In [0]:
import spacy
nlp = spacy.load("en")

In [0]:
text =  "I’ve been called many things in my life, but never an optimist. That was fine by me. I believed pessimists lived in a constant state of pleasant surprise: if you always expected the worst, things generally turned out better than you imagined. The only real problem with pessimism, I figured, was that too much of it could accidentally turn you into an optimist."

In [0]:
import nltk #this part is needed on colab.
nltk.download('punkt')
nltk.download('stopwords')
#----------------------------------------

# Tokenizing sentences
from nltk.tokenize import sent_tokenize

# Tokenizing words
from nltk.tokenize import word_tokenize

import pandas as pd
pd.set_option('display.max_colwidth', -1) #to see more text

import numpy as np
import seaborn as sns

import itertools
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# Tokenizing Tweets made easy!
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 

In [0]:
# Let's get our stences.
# Note that the full-stops at the end of each sentence are still there

sentences = sent_tokenize(text)
print(sentences)

['I’ve been called many things in my life, but never an optimist.', 'That was fine by me.', 'I believed pessimists lived in a constant state of pleasant surprise: if you always expected the worst, things generally turned out better than you imagined.', 'The only real problem with pessimism, I figured, was that too much of it could accidentally turn you into an optimist.']


In [0]:
# Use word_tokenize to tokenize the third sentence: tokenized_sent
tokenized_sent = word_tokenize(sentences[1])

# Make a set of unique tokens in the entire scene: unique_tokens
unique_tokens = set(word_tokenize(text))

print(tokenized_sent)
print(unique_tokens)

['That', 'was', 'fine', 'by', 'me', '.']
{'life', 'expected', 'The', 'much', 'optimist', 'in', '’', 'but', 'a', 'an', 'turned', 'could', 'pessimists', 'only', 'constant', 'lived', 'believed', 'out', 'pleasant', 'better', 'by', 'I', ',', 'been', 'pessimism', 'into', 'my', 'was', 'of', 'imagined', 'you', 'that', 'problem', 'state', 'surprise', 'than', 've', 'if', 'fine', 'things', 'accidentally', ':', 'it', 'worst', 'figured', 'the', 'called', 'many', 'real', 'generally', 'That', 'never', 'too', '.', 'turn', 'me', 'always', 'with'}


In [0]:
[word.lower() for word in word_tokenize(text) if word not in stop_words and word.isalnum()]

['i',
 'called',
 'many',
 'things',
 'life',
 'never',
 'optimist',
 'that',
 'fine',
 'i',
 'believed',
 'pessimists',
 'lived',
 'constant',
 'state',
 'pleasant',
 'surprise',
 'always',
 'expected',
 'worst',
 'things',
 'generally',
 'turned',
 'better',
 'imagined',
 'the',
 'real',
 'problem',
 'pessimism',
 'i',
 'figured',
 'much',
 'could',
 'accidentally',
 'turn',
 'optimist']

# Part 2: Baby boomer tweets - applying tokens and counts to text.

This section is an attempt to model the 10 most popular topics in the tweets that tag baby boomer. 

These topics are extracted from the text portion rather than from the tags themselves

In [0]:
# download and open some Trump tweets from trump_tweet_data_archive

boomer_tweets_df = pd.read_json('https://github.com/CALDISS-AAU/sdsphd19_coursematerials/raw/master/data/tweets_boomer.zip')
# boomer_tweets_df.head()

In [0]:
# Reset index (not really needed but why not)
boomer_tweets_df = boomer_tweets_df.set_index(pd.to_datetime(boomer_tweets_df.created_at))

In [0]:
boomer_tweets_df.info()

# Cleaning boomer tweets to text


In [0]:
!pip install tweet-preprocessor


In [0]:
import preprocessor as p

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [0]:
p.set_options(p.OPT.URL, 	p.OPT.HASHTAG, p.OPT.EMOJI, p.OPT.MENTION, 	p.OPT.RESERVED, 	p.OPT.SMILEY, 	p.OPT.NUMBER)
# [p.clean(tweet) for tweet in boomer_tweets_df['tweet']]

In [0]:
boomer_tweets_clean = boomer_tweets_df.tweet.map(p.clean)

In [0]:
boomer_tweets_df.head()

In [0]:
tknzr = TweetTokenizer(strip_handles=True)

boomer_tweets_clean = boomer_tweets_clean.map(lambda t: [tok.lower() for tok in word_tokenize(t) if tok not in stop_words and tok.isalpha()])

In [0]:
boomer_tweets_clean = boomer_tweets_clean.map(lambda t:  " ".join(t))

In [0]:
boomer_tweets_clean

In [0]:
tokens = []

for x in nlp.pipe(boomer_tweets_clean):
  proj_tok = [token.lemma_.lower() for token in x if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] and not token.is_stop] 
  tokens.append(proj_tok)

In [0]:
tokens


In [0]:
boomer_tweets_df['tokens'] = tokens

In [0]:
boomer_tweets_df.head()

In [0]:
!pip install -qq -U gensim

In [0]:
# Import the dictionary builder
from gensim.corpora.dictionary import Dictionary

In [0]:
# Create a Dictionary from the articles: dictionary
dictionary = Dictionary(boomer_tweets_df['tokens'])

In [0]:
# filter out low-frequency / high-frequency stuff, also limit the vocabulary to max 1000 words
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)

In [0]:
# construct corpus using this dictionary
corpus = [dictionary.doc2bow(doc) for doc in boomer_tweets_df['tokens']]

In [0]:
# That's how the corpus looks
corpus[:100]

In [0]:
# let's fist install this nice visualizer
!pip install -qq pyLDAvis

In [0]:
# and import it
import pyLDAvis.gensim
%matplotlib inline
pyLDAvis.enable_notebook()

In [0]:
# we'll use the faster multicore version of LDA

from gensim.models import LdaMulticore

In [0]:
# Training the model (makes some mess atm due to version clashes)

lda_model = LdaMulticore(corpus, id2word=dictionary,  num_topics=10, workers = 4, passes=10)

In [0]:
# Check out topics
lda_model.print_topics(-1)

In [0]:
# Let's try to visualize
lda_display = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics=False)


/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [0]:
# Let's Visualize
pyLDAvis.display(lda_display)

# Plotting most common by day

In [0]:
# Get columns of interest into a new DF

tweets_boomer_tokens = boomer_tweets_df[['tweet', 'tokens', 'id','created_at']]

In [0]:
# Get most common hashtags

most_common = Counter(itertools.chain(*tweets_boomer_tokens.tokens)).most_common(11)

In [0]:
most_common

[('boomer', 2600),
 ('people', 788),
 ('generation', 689),
 ('old', 680),
 ('millennial', 544),
 ('time', 462),
 ('thing', 443),
 ('meme', 414),
 ('way', 406),
 ('good', 335),
 ('young', 330)]

In [0]:
# Isolate the hashtags only
most_common = [x for x,y in most_common][1:]

In [0]:
# Create new columns (one for each hashtag) and fill with 0

for col in most_common:
  tweets_boomer_tokens[col] = 0

In [0]:
# Iterate over hashtags and fill in the corresponding columns with 1 if hashtag present in a tweet
for tokens in most_common:
  tweets_boomer_tokens[tokens] = tweets_boomer_tokens['tokens'].map(lambda t: 1 if tokens in t else 0)

In [0]:
# Import plotting stuff
import seaborn as sns
sns.set_style("whitegrid")
sns.set(rc={'figure.figsize':(10,10)})

In [0]:
# count up occurence of hashtags in timeframe of interest

tweets_boomer_tokens.resample('D')[most_common].sum().plot()

# Processing an academic article

This is an attempt to apply the tools above to an academic article to gain insight into the key focus of the research.

In [0]:
!pip install PyPDF2
!pip install tika

In [0]:
#import requests

#url = 'http://www.levyinstitute.org/pubs/wp_891.pdf'

#pdf_file_content = requests.get(url).content

#pdf_file = requests.get(url).content

In [0]:

from tika import parser
import PyPDF2 as pyPdf

raw = parser.from_file('http://www.levyinstitute.org/pubs/wp_891.pdf')
content = raw['content']

2019-12-02 16:21:49,186 [MainThread  ] [INFO ]  Retrieving http://www.levyinstitute.org/pubs/wp_891.pdf to /tmp/pubs-wp_891.pdf.


In [0]:
# raw["metadata"]

In [0]:
text_content = " ".join(content.replace(u"\xa0", " ").strip().split())

In [0]:
text_content

'Stock-flow Consistent Macroeconomic Models: a Survey Working Paper No. 891 Stock-flow Consistent Macroeconomic Models: A Survey by Michalis Nikiforos and Gennaro Zezza Levy Economics Institute of Bard College May 2017 The Levy Economics Institute Working Paper Collection presents research in progress by Levy Institute scholars and conference participants. The purpose of the series is to disseminate ideas to and elicit comments from academics and professionals. Levy Economics Institute of Bard College, founded in 1986, is a nonprofit, nonpartisan, independently funded research organization devoted to public service. Through scholarship and economic research it generates viable, effective public policy responses to important economic problems that profoundly affect the quality of life in the United States and abroad. Levy Economics Institute P.O. Box 5000 Annandale-on-Hudson, NY 12504-5000 http://www.levyinstitute.org Copyright © Levy Economics Institute 2017 All rights reserved ISSN 15

In [0]:
import pandas as pd
import numpy as np

In [0]:
# Here is a long article text - again
article = text_content

In [0]:
len(article)

148062

In [0]:
article.find("REFERENCES")

105484

In [0]:
references_text = article[105480:]
article_text = article[:105480]

In [0]:
# Separate sentences as a separate list
sentences = sent_tokenize(article_text)
reference_sentences = sent_tokenize(references_text)
#print(sentences)

In [0]:
# create reference list tokens
sent_tokens = word_tokenize(sentences[1])

# Create sentence tokens
reference_sent_tokens = word_tokenize(reference_sentences[1])

# Make a set of unique tokens in the entire article: unique_tokens
word_tokens = set(word_tokenize(article_text))

In [0]:
!pip install tweet-preprocessor
import preprocessor as p

In [0]:
p.set_options(p.OPT.URL, 	p.OPT.HASHTAG, p.OPT.EMOJI, p.OPT.MENTION, 	p.OPT.RESERVED, 	p.OPT.SMILEY, 	p.OPT.NUMBER)

In [0]:
# Clean the article section using the twee cleaner
sent_text_df = pd.DataFrame(sent_tokens)
sent_text_df[0]
sent_tokens = list(reference_text_df[0].map(p.clean))

# Clean the references section using the tweet cleaner
reference_text_df = pd.DataFrame(reference_sent_tokens)
reference_text_df[0]
references_text_clean = list(reference_text_df[0].map(p.clean))

# Clean the individual words list using the tweet cleaner
word_text_df = pd.DataFrame(word_tokens)
word_text_df[0]
word_tokens = list(word_text_df[0].map(p.clean))


In [0]:
word_tokens

In [0]:
#print(tokenized_sent)
#print(unique_tokens)
print(type(tokenized_sent))
print(type(word_tokens))


<class 'list'>
<class 'set'>


In [0]:
# Additional exceptions for tokens

list_of_random_exclusions = ['k', '-', '—', '2012a']

# Cleaning the sentences and individual words of the article


In [0]:
tokens_sent = []

for x in nlp.pipe(sent_tokens):
  proj_tok = [token.lemma_.lower() for token in x if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] and not token.is_stop and token.pos_ not in list_of_random_exclusions] 
  tokens_sent.append(proj_tok)

In [0]:
tokens_words = []

for x in nlp.pipe(word_tokens):
  proj_tok = [token.lemma_.lower() for token in x if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] and not token.is_stop and token.pos_ not in list_of_random_exclusions] 
  tokens_words.append(proj_tok)

In [0]:
tokens_sent

In [0]:
tokens_words

## Load the graphical interface and dictionary generator


In [0]:
!pip install -qq -U gensim

In [0]:
# Import the dictionary builder
from gensim.corpora.dictionary import Dictionary

In [0]:
# Create a Dictionary from the articles: dictionary
dictionary_sent = Dictionary(tokens_sent)
dictionary_words = Dictionary(tokens_words)

In [0]:
# filter out low-frequency / high-frequency stuff, also limit the vocabulary to max 1000 words
dictionary_sent.filter_extremes(no_below=3, no_above=0.5, keep_n=1000)
dictionary_words.filter_extremes(no_below=3, no_above=0.5, keep_n=1000)

In [0]:
# construct corpus using this dictionary
corpus_sent = [dictionary_sent.doc2bow(doc) for doc in tokens_sent]
corpus_words = [dictionary_words.doc2bow(doc) for doc in tokens_words]

In [0]:
# That's how the corpus looks
# corpus_sent[:]
corpus_words[:100]

In [0]:
# and import it
import pyLDAvis.gensim
%matplotlib inline
pyLDAvis.enable_notebook()

In [0]:
# we'll use the faster multicore version of LDA

from gensim.models import LdaMulticore

In [0]:
# Training the model (makes some mess atm due to version clashes)

lda_model = LdaMulticore(corpus_words, id2word=dictionary_words,  num_topics=10, workers = 4, passes=10)

In [0]:
# Check out topics
lda_model.print_topics(-1)

In [0]:
# Let's try to visualize
lda_display = pyLDAvis.gensim.prepare(lda_model, corpus_words, dictionary_words, sort_topics=False)


of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.
 [_prepare.py:257]


In [0]:
# Let's Visualize
pyLDAvis.display(lda_display)